# Deploying a Model as a Web Service

It's no good being able to create an accurate model if you can't deploy it for use in an application or service. In this notebook, we'll explore the *Azure Machine Learning Service* and the associated *Azure Machine Learning SDK*; which together enable you to train, deploy, and manage machine learning models at scale.

To use Azure Machine Learning, you're going to need an Azure subscription. If you don't already have one, you can sign up for a free trial at https://azure.microsoft.com/Account/Free.

*Note: Azure Machine Learning provides a whole range of functionality to help you through the entire lifecycle of model development, training, evaluation, deployment, and management. We're going to focus on using it to deploy a trained model; but you can use it to do much, much more!*

## A Brief Introduction to Containers
When you access a web site or a software service across the internet, you're probably dimly aware that somewhere, the code for the service is hosted on a *server*. We tend to think of servers as being physical computers, but in recent years there's been a growth in *virtualization* technologies so that a computer can be virtualized in software, and multiple *virtual machines* can be hosted on a single physical server.

Virtual machines (VMs) are useful - in fact, the Azure Data Science Virtual Machine (DSVM) is a good example of a VM that enables you to provision a computer that contains the operating system (OS) and all the software applications you need to work with data and build machine learning models, and then you can delete the VM when you're finished with it so that you only pay for what you use - very cool!

However, it seems wasteful to provision a complete virtual machine, including the full OS and applications, just to host a simple software service - especially if you need to support multiple services, each one consuming its own VM. *Containers* are an evolutionary step beyond VMs. They contain only the OS components that are required for the specific software service they need to host. This makes them very small compared to full VMs, which in turn means that they're portable, and quick to deploy and start up.

Containers themselves are hosted in a container environment that provides all the common services and OS functionality they require. During development, this environment is often a locally installed system called *Docker*. When hosting a service in the cloud however, you can use container services such as *Azure Container Instances* (ACI), which is useful for lightweight hosting and testing of containerized services; or *Azure Kubernetes Services*, which provides a scalable and highly-available environment for managing clusters of containers, based on the industry standard *Kubernetes* container hosting platform.

In the rest of this notebook, we'll examine how you can use Azure Machine Learning Services to prepare a container image for your machine learning model, and deploy your model as a containerized web service that can be consumed by other applications that connect to it over an HTTP REST endpoint.

## Train a classification model
Let's start by training a simple classification model so that we have something to deploy.
If you've viewed the previous sample notebooks in this hack, this should be pretty familiar - we're going to use PyTorch to train a simple shape classifier.

First, we'll install the latest version of PyTorch and import the libraries we'll use to train and test the model locally.

In [ ]:
# Install PyTorch
import sys
! {sys.executable} -m pip install --upgrade torch
! {sys.executable} -m pip install --upgrade torchvision

# Import PyTorch libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

print("Libraries imported - ready to use PyTorch", torch.__version__)

# Other libraries we'll use
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

Next we'll generate some images of geometric shapes with which to train and validate the model.

In [ ]:
# Function to create a random image (of a square, circle, or triangle)
def create_image (size, shape):
    from random import randint
    import numpy as np
    from PIL import Image, ImageDraw
    
    xy1 = randint(10,40)
    xy2 = randint(60,100)
    col = (randint(0,200), randint(0,200), randint(0,200))

    img = Image.new("RGB", size, (255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    if shape == 'circle':
        draw.ellipse([(xy1,xy1), (xy2,xy2)], fill=col)
    elif shape == 'square':
        draw.rectangle([(xy1,xy1), (xy2,xy2)], fill=col)
    else: # triangle
        draw.polygon([(xy1,xy1), (xy2,xy2), (xy2,xy1)], fill=col)
    del draw
    
    return np.array(img)
    return np.array(img)

# function to create a dataset of images
def generate_image_data (classes, size, cases, img_dir):
    import os, shutil
    from PIL import Image
    
    if os.path.exists(img_dir):
        replace_folder = input("Image folder already exists. Enter Y to replace it (this can take a while!). \n")
        if replace_folder == "Y":
            print("Deleting old images...")
            shutil.rmtree(img_dir)
        else:
            return # Quit - no need to replace existing images
    os.makedirs(img_dir)
    print("Generating new images...")
    i = 0
    while(i < (cases - 1) / len(classes)):
        if (i%25 == 0):
            print("Progress:{:.0%}".format((i*len(classes))/cases))
        i += 1
        for classname in classes:
            img = Image.fromarray(create_image(size, classname))
            saveFolder = os.path.join(img_dir,classname)
            if not os.path.exists(saveFolder):
                os.makedirs(saveFolder)
            imgFileName = os.path.join(saveFolder, classname + str(i) + '.jpg')
            try:
                img.save(imgFileName)
            except:
                try:
                    # Retry (resource constraints in Azure notebooks can cause occassional disk access errors)
                    img.save(imgFileName)
                except:
                    # We gave it a shot - time to move on with our lives
                    print("Error saving image", imgFileName)
                    
# Function to ingest data using training and test loaders
def load_dataset(data_path):
    # Load all of the images
    transformation = transforms.Compose([
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transformation
    )
    
    
    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=50,
        num_workers=0,
        shuffle=False
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=50,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader
            
# Our classes will be circles, squares, and triangles
classnames = ['circle', 'square', 'triangle']

# All images will be 128x128 pixels
img_size = (128,128)

# We'll store the images in a folder named 'shapes'
folder_name = 'shapes'

# Generate 1200 random images.
generate_image_data(classnames, img_size, 1200, folder_name)

# Now load the images from the shapes folder
print("Loading image files in %s folder..." % folder_name)
data_path = folder_name + '/'

# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(data_path)

# Get the class names
classes = os.listdir(data_path)
classes.sort()
print("Data loaded. \n Class names: ", classes)

Now we'll define and train the model.

In [ ]:
# Create a neural net class
class Net(nn.Module):
    # Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        
        # Our images are RGB, so input channels = 3. We'll apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # We'll apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # So our feature tensors are now 32 x 32, and we've generated 24 of them
        # We need to flatten these and feed them to a fully-connected layer
        # to map them to  the probability for each class
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=num_classes)

    def forward(self, x):
        # Use a relu activation function after layer 1 (convolution 1 and pool)
        x = F.relu(self.pool(self.conv1(x)))
      
        # Use a relu activation function after layer 2 (convolution 2 and pool)
        x = F.relu(self.pool(self.conv2(x)))
        
        # Select some features to drop after the 3rd convolution to prevent overfitting
        x = F.relu(self.drop(x))
        
        # Only drop the features if this is a training pass
        x = F.dropout(x, training=self.training)
        
        # Flatten
        x = x.view(-1, 32 * 32 * 24)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        # Return class probabilities via a softmax function 
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        # Use the CPU or GPU as appropriate
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
        output = model(data)
        
        # Get the loss
        loss = loss_criteria(output, target)
        
        # Keep a running total
        train_loss += loss.item()
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        # Print metrics for every 10 batches so we see some progress
        if batch_idx % 10 == 0:
            print('Training set [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    # return average loss for the epoch
    return train_loss / len(train_loader.dataset)
            
            
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += loss_criteria(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    # Calculate the average loss and total accuracy for this epoch
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return test_loss
    
# Now Use the train and test functions to train and test the model    

device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"
print('Training on', device)

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=len(classnames)).to(device)

# Use an "Adam" optimizer to adjust weights
# (see https://pytorch.org/docs/stable/optim.html#algorithms for details of supported algorithms)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.CrossEntropyLoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []
validation_loss = []

# Train over 5 epochs (in a real scenario, you'd likely use many more)
epochs = 5
for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)

## Save and test the model locally
OK, so we now have a trained shape classification model. Let's save it as a local file (well, local to the Azure Notebooks library anyway), and then load and test it; just to satisfy ourselves that it works:

In [ ]:
# Function to predict the class of an image
def predict_image(classifier, image_array):
    import torch.utils.data as utils
    import numpy as np
    
    # Set the classifer model to evaluation mode
    classifier.eval()
    
    # Apply the same transformations as we did for the training images
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Preprocess the image
    image_tensor = torch.stack([transformation(image).float() for image in image_array])

    # Turn the input into a Variable
    input_features = Variable(image_tensor)

    # Predict the class of the image
    predictions = classifier(input_features)
    
    predicted_classes = []
    for prediction in predictions.data.numpy():
        class_idx = np.argmax(prediction)
        predicted_classes.append(classes[class_idx])
    return np.array(predicted_classes)

# Save the model weights
model_file = 'shape-classifier.pth'
torch.save(model.state_dict(), model_file)
print("Model saved.")

# Delete the existing model variable
del model

# Create a new model and load the weights
model = Net()
model.load_state_dict(torch.load(model_file))
print("New model created from saved weights")


# Now let's try it with a new image
from random import randint

# Create a random test image
img = create_image ((128,128), classes[randint(0, len(classes)-1)])
plt.imshow(img)

# Create an array of (1) images to match the expected input format
image_array = img.reshape(1, img.shape[0], img.shape[1], img.shape[2]).astype('float32')

predicted_classes = predict_image(model, image_array)
print(predicted_classes[0])

It looks as though we have a working model. Now we're ready to use Azure Machine Learning to deploy it as a web service.

## Create an Azure Machine Learning workspace

To use Azure Machine Learning, you'll need to create a workspace in your Azure subscription.

Your Azure subscription is identified by a subscription ID. To find this:
1. Sign into the Azure portal at https://portal.azure.com.
2. On the menu tab on the left, click &#128273; **Subscriptions**.
3. View the list of your subscriptions and copy the ID for the subscription you want to use.
4. Paste the subscription ID into the code below, and then run the cell to set the variable - you will use it later.

In [ ]:
# Replace YOUR_SUBSCRIPTION_ID in the following variable assignment:
SUBSCRIPTION_ID = 'YOUR_SUBSCRIPTION_ID'

To deploy the model file as a web service, we'll use the Azure Machine Learning SDK.

> Note: the Azure Machine Learning SDK is installed by default in Azure Notebooks and the Azure Data Science Virtual Machine, but you may want to ensure that it's upgraded to the latest version. If you're using your own Python environment, you'll need to install it using the instructions in the [Azure Machine Learning documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)*

In [ ]:
import sys
! {sys.executable} -m pip install azureml-sdk --upgrade

import azureml.core
print(azureml.core.VERSION)

To manage the deployment, we need an Azure ML workspace. Create one in your Azure subscription by running the following cell. If you're signed into Azure notebooks using the same credentials as your Azure subscription, you may be prompted to grant this notebooks project permission to use your Azure credentials. Otherwise, you'll be prompted to authenticate by entering a code at a given URL, so just click the link that's displayed and enter the specified code.

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name='aml_workspace', # or another name of your choosing
                      subscription_id=SUBSCRIPTION_ID,
                      resource_group='aml_resource_group', # or another name of your choosing
                      create_resource_group=True,
                      location='eastus2' # or other supported Azure region
                     )

Now that you have a workspace, you can save the configuration so you can reconnect to it later.

In [ ]:
from azureml.core import Workspace

# Save the workspace config
ws.write_config()

# Reconnect to the workspace (if you're not already signed in, you'll be prompted to authenticate with a code as before)
ws = Workspace.from_config()

## Create a *scoring* file
Your web service will need some Python code to load the input data, get the model, and generate and return a prediction. We'll save this code in a *scoring* file that will be deployed to the web service:

In [ ]:
%%writefile score_pytorch.py

# create a scoring script that loads and infers from the model
import json
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from azureml.core.model import Model

def init():
    try:
        global model
        MODEL_NAME = 'shape-classifier.pth'
        # retieve the local path to the model using the model name
        MODEL_PATH = Model.get_model_path(MODEL_NAME)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        model = Net()
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

# REST API served by Azure ML supports json input
def run(json_data):
    try:
        data = np.array(json.loads(json_data)['data']).astype('float32')
        predictions = predict_image(model, data)
        return json.dumps(predictions.tolist())
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

# Function to predict
def predict_image(classifier, image_array):
    import torch
    import torch.utils.data as utils
    from torchvision import transforms
    from torch.autograd import Variable
    import numpy
    
    classifier.eval()
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    image_tensor = torch.stack([transformation(image).float() for image in image_array])

    input_features = Variable(image_tensor)
    predictions = classifier(input_features)
    
    classnames = ['circle', 'square', 'triangle']
    
    predicted_classes = []
    for prediction in predictions.data.numpy():
        class_idx = np.argmax(prediction)
        predicted_classes.append(classnames[class_idx])
    return np.array(predicted_classes)
    
# Define the Net class as used for training so we can load the trained weights
class Net(nn.Module):
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.drop = nn.Dropout2d(p=0.2)
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=num_classes)
    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.drop(x))
        x = F.dropout(x, training=self.training)
        x = x.view(-1, 32 * 32 * 24)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

## Create an *environment* file
The web service will be hosted in a container, and the container will need to install any Python dependencies when it gets initialized. In this case, our scoring code requires the **torch** and **torchvision** Python libraries, so we'll create a .yml file that tells the container host to install these into the environment along with the default libraries used by Azure ML.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("pytorch")
myenv.add_conda_package("torchvision")
myenv.add_channel("pytorch")

env_file = "env_pytorch.yml"

with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

with open(env_file,"r") as f:
    print(f.read())

## Define a container image
We're going to deploy the web service as a container, so we need to define a container image that includes our scoring file and denvironment dependencies.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score_pytorch.py",
                                                  runtime = "python",
                                                  conda_file = env_file,
                                                  description = "Container image for shape classification",
                                                  tags = {"data": "shapes", "type": "classification"}
                                                 )
print(image_config.description)

## Define the web service deployment configuration
We're going to deploy the containerized web service in the Azure Container Instance (ACI) service, so we need to specify the deployment configuration.

In [ ]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "shapes", "type": "classification"},
                                               description = 'shape classification service')
print(aci_config.description)

## Deploy the web service 
Now we're ready to deploy. We'll deploy the container a service named **aci-pytorch-shape-svc**.
The deployment process includes the following steps:
1. Register the model file in the Azure Machine Learning service (this also uploads the local model file to your Azure Machine Learning service so it can be deployed to a container)
2. Create a container image for the web service, based on the configuration specified previously. This image will be used to instantiate the service.
3. Create a service by deploying the container image (in this case to ACI - other hosts are available!)
4. Verify the status of the deployed service.

> For more information about Azure Container Instances, see https://azure.microsoft.com/en-us/services/container-instances.

This will take some time. When deployment has completed successfully, you'll see a status of **Healthy**.

In [ ]:
from azureml.core.webservice import Webservice

service_name = 'aci-pytorch-shape-svc'
service = Webservice.deploy(deployment_config = aci_config,
                                image_config = image_config,
                                model_paths = [model_file],
                                name = service_name,
                                workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

## Use the web service
With the service deployed, now we can test it by using it to predict the shape of a new image.

In [ ]:
import json
from random import randint

# Create a random test image
img = create_image ((128,128), classes[randint(0, len(classes)-1)])
plt.imshow(img)

# Modify the image data to create an array of 1 image, matching the format of the training features
input_array = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

# Convert the array to JSON format
input_json = json.dumps({"data": input_array.tolist()})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
classname = json.loads(predictions)[0]
print('The image is a', classname)

You can also send a batch of images to the service, and get back a prediction for each one.

In [ ]:
import json
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline

# Create three random test images
fig = plt.figure(figsize=(6, 6))
images = []
i = 0
while(i < 3):  
    # Create a new image
    img = create_image((128,128), classes[randint(0, len(classes)-1)])
    # Plot the image
    a=fig.add_subplot(1,3,i + 1)
    imgplot = plt.imshow(img)
    # Add the image to an array to be submitted as a batch
    images.append(img.tolist())
    i += 1

# Convert the array to JSON format
input_json = json.dumps({"data": images})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes
print(json.loads(predictions))

### Using the Web Service from Other Applications
The code above uses the Azure ML SDK to connect to the containerized web service and use it to generate predictions from your image classification model. In production, the model is likely to be consumed by business applications that make HTTP requests to the web service.

Let's determine the URL to which these applications must submit their requests:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Now that we know the endpoint URI, an application can simply make an HTTP request, sending the image data in JSON (or binary) format, and receive back the predicted class(es).

In [ ]:
import requests
import requests
import json
from random import randint

# Create a random test image
img = create_image ((128,128), classes[randint(0, len(classes)-1)])
plt.imshow(img)

# Create an array of (1) images to match the expected input format
image_array = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": image_array.tolist()})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
print(json.loads(predictions.content))

## Deleting the Service
When we're finished with the service, we can delete it to avoid incurring charges.

In [ ]:
service.delete()
print("Service deleted.")

And if you're finished with the workspace, you can delete that too

In [ ]:
rg = ws.resource_group
ws.delete()
print("Workspace deleted. You should delete the '%s' resource group in your Azure subscription." % rg)

## Learn more
Take a look at the Azure Machine Learning documentation at https://docs.microsoft.com/en-us/azure/machine-learning/service/.